In [1]:
import os, sys

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from tensorflow import keras

from scripts.data_generator import DataGenerator, LoadData
from scripts.config import train_set, index_map, LANDMARKS, DATAPATH

In [ ]:
# Training with parquet files

data_size = len(train_set)
list_IDs = np.arange(data_size)
np.random.shuffle(list_IDs)
val_start = int(data_size*.95)
batch_size = 128

# Generators
train_gen = DataGenerator(list_IDs[:val_start], train_set,
                          index_map, batch_size=batch_size)
val_gen = DataGenerator(list_IDs[val_start:], train_set,
                        index_map, batch_size=batch_size)

input_shape = (160, 85, 3)
model = keras.applications.EfficientNetB0(
    weights=None,
    input_shape=input_shape,
    classes=250
)
model.compile(optimizer='Adam',
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])

# checkpoint
checkpoint = keras.callbacks.ModelCheckpoint('best_model.h5',
                                                monitor='accuracy', mode='max',
                                                verbose=1, save_best_only=True)
es = keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                      verbose=1, patience=200)
callbacks_list = [es, checkpoint]

# Train model on dataset
model.fit(x=train_gen,
          validation_data=val_gen,
          epochs=20,
          callbacks=callbacks_list)

In [ ]:
# Create custom dataset

data_size = len(train_set)
input_shape = (60, 85)
X = np.empty((data_size, input_shape[0], input_shape[1], 3))
y = np.empty((data_size, 250), dtype=int)
dataloader = LoadData()

for index, row in train_set.iterrows():
    data = dataloader.load_relevant_data_subset(DATAPATH + row.path)
    data = np.nan_to_num(data[:, LANDMARKS, :], 0)
    data = tf.image.resize(data, size=input_shape, method='nearest')
    X[index, ] = data.numpy()
    y[index, ] = keras.utils.to_categorical(index_map[row.sign], num_classes=250)

print(f'Data size: {len(X)}\nX shape: {X[0].shape}\ny shape: {y[0].shape}')


np.save('data/X_data', X)
np.save('data/y_data', y)

In [2]:
class DataSequence(keras.utils.Sequence):
    def __init__(self, X, y, batch_size=32, shuffle=True):
        self.X, self.y = X, y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        return self.X[indexes], self.y[indexes]
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.X))
        if self.shuffle is True:
            np.random.shuffle(self.indexes)

In [6]:
# Training with custom dataset
X = np.load('data/X_data.npy')
y = np.load('data/y_data.npy')

model = keras.applications.EfficientNetB1(
    weights=None,
    input_shape=X.shape[1:],
    classes=y.shape[1]
)
model.compile(optimizer='Adam',
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])

# training callbacks
checkpoint = keras.callbacks.ModelCheckpoint(
    'best_model_pool.h5', monitor='val_loss', mode='min',
    verbose=1, save_best_only=True
    )
es = keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=10
    )
cb_list = [es, checkpoint]

# Create generators
data_size = len(X)
val_start = int(data_size*.95)
batch_size = 32

train_gen = DataSequence(X[:val_start], y[:val_start], batch_size=batch_size)
val_gen = DataSequence(X[val_start:], y[val_start:], batch_size=batch_size)

# Train model on dataset
model.fit(x=train_gen,
          validation_data=val_gen,
          epochs=40,
          callbacks=cb_list)

Epoch 1/40
2804/2804 [==============================] - ETA: 0s - loss: 5.3502 - accuracy: 0.0101
Epoch 1: val_loss improved from inf to 5.17368, saving model to best_model_pool.h5
2804/2804 [==============================] - 213s 66ms/step - loss: 5.3502 - accuracy: 0.0101 - val_loss: 5.1737 - val_accuracy: 0.0128
Epoch 2/40
2804/2804 [==============================] - ETA: 0s - loss: 4.3938 - accuracy: 0.0663
Epoch 2: val_loss improved from 5.17368 to 4.04500, saving model to best_model_pool.h5
2804/2804 [==============================] - 183s 65ms/step - loss: 4.3938 - accuracy: 0.0663 - val_loss: 4.0450 - val_accuracy: 0.1084
Epoch 3/40
2804/2804 [==============================] - ETA: 0s - loss: 3.4313 - accuracy: 0.2057
Epoch 3: val_loss improved from 4.04500 to 2.89428, saving model to best_model_pool.h5
2804/2804 [==============================] - 184s 65ms/step - loss: 3.4313 - accuracy: 0.2057 - val_loss: 2.8943 - val_accuracy: 0.3087
Epoch 4/40
2804/2804 [===================

In [7]:
class Preprocess(tf.keras.layers.Layer):
    def __init__(self, point_landmarks=LANDMARKS, shape=(60,85), **kwargs):
        super().__init__(**kwargs)
        self.point_landmarks = point_landmarks
        self.shape = shape

    def call(self, inputs):
        x = tf.gather(inputs, LANDMARKS, axis=1)
        x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
        x  = tf.image.resize(x, size=(self.shape[0],85), method='nearest')
        return tf.reshape(x, (1,self.shape[0],self.shape[1],3))
    
class InferenceModel(tf.Module):
    def __init__(self, islr_model, shape=(60,85)):
        super(InferenceModel, self).__init__()
        self.islr_model = islr_model
        self.shape = shape
        self.prep_inputs = Preprocess(shape=self.shape)
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs):
        x = self.prep_inputs(tf.cast(inputs, dtype=tf.float32))
        outputs = self.islr_model(x) 
        return {'outputs': outputs}

In [8]:
islr_model = keras.models.load_model('best_model_pool.h5')
inference_model = InferenceModel(islr_model, shape=X.shape[1:3])

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp7jbpyh2p/assets


INFO:tensorflow:Assets written to: /tmp/tmp7jbpyh2p/assets
